# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [1]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
!gdown --id '149paISvxCXDlr-720UEzseQA-y53rZxN' --output food-11.zip 

# Dropbox
# !wget "https://www.dropbox.com/s/7yl5rra84ia0k8f/food-11.zip?dl=0" -O food-11.zip

# MEGA
# !wget https://megatools.megous.com/builds/megatools-1.10.3.tar.gz
# !tar -zxvf /content/megatools-1.10.3.tar.gz
# !sudo apt-get install libtool libglib2.0-dev gobject-introspection libgmp3-dev nettle-dev asciidoc glib-networking openssl libcurl4-openssl-dev libssl-dev
# %cd megatools-1.10.3/
# !./configure make
# !sudo make install
# %cd /content/
# !megadl 'https://mega.nz/file/FdlygByK#QQ5LP71MMjeXrXwoXM2qlygUsJ1D-6d5fMhj5gyi2Vc'

# Unzip the dataset.
# This may take some time.
!unzip -n food-11.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: food-11/training/unlabeled/00/5451.jpg  
  inflating: food-11/training/unlabeled/00/5454.jpg  
  inflating: food-11/training/unlabeled/00/5459.jpg  
  inflating: food-11/training/unlabeled/00/5462.jpg  
  inflating: food-11/training/unlabeled/00/5470.jpg  
  inflating: food-11/training/unlabeled/00/5479.jpg  
  inflating: food-11/training/unlabeled/00/5497.jpg  
  inflating: food-11/training/unlabeled/00/5498.jpg  
  inflating: food-11/training/unlabeled/00/5502.jpg  
  inflating: food-11/training/unlabeled/00/5505.jpg  
  inflating: food-11/training/unlabeled/00/5515.jpg  
  inflating: food-11/training/unlabeled/00/5520.jpg  
  inflating: food-11/training/unlabeled/00/5530.jpg  
  inflating: food-11/training/unlabeled/00/5531.jpg  
  inflating: food-11/training/unlabeled/00/5541.jpg  
  inflating: food-11/training/unlabeled/00/5548.jpg  
  inflating: food-11/training/unlabeled/00/5549.jpg  
  inflating: food-11/training/unlabeled/00/5565.jpg  
  infla

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [2]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm import tqdm
import torchvision.models as models

In [3]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Parameters
same_seeds(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64
n_epochs = 4096
earlystop = 4096
threshold = 0.99
do_semi = False
Scheduler = "Cosine"
model_path = "model_window_0.ckpt"
# current_best_model_path = "current_best_model_79.ckpt"
saving_path = "pred.csv"

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [4]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.Resize((128, 128)),
    transforms.RandomRotation(30),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size=(128,128)),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.ColorJitter(contrast=0.5, brightness=0.5),
    transforms.RandomErasing(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [5]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
tfm_unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
tfm_test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
MyPseudoLabel_unlabeled_set = []
MyPseudoLabel_test_set = []

In [7]:
def semi_unlabeled_set(index):
    # print("ckpt 0: ",len(MyPseudoLabel_unlabeled_set))
    return MyPseudoLabel_unlabeled_set[index]

def semi_test_set(index):
    # print("ckpt 1: ",len(MyPseudoLabel_test_set))
    return MyPseudoLabel_test_set[index]

## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(

            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.Conv2d(64, 64, 3, 1, 1),
            # nn.BatchNorm2d(64),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.Conv2d(128, 128, 3, 1, 1),
            # nn.BatchNorm2d(128),
            # nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

        )
        self.fc_layers = nn.Sequential(

            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 1000),
            nn.ReLU(),
            nn.BatchNorm1d(1000),
            nn.Dropout(0.5),
            
            nn.Linear(1000, 11)

        )

    def forward(self, x):
        # ------ #
        
        # x = self.conv((x.view(-1,1,128,128))).view(64,-1,128,128)


        # ------ #
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)
        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)
        # print(x.shape)
        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [9]:
def get_pseudo_labels_0(dataset, model):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    model.eval()
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    indice = []
    label = []
    idx = 0
    count = torch.zeros((11))
    # print("ckpt 1: ",len(MyPseudoLabel))
    for batch in dataloader:
        img, _= batch
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        max_prob, arg = torch.max(probs,1)
        for i in range(len(probs)):
            if(max_prob[i] >= threshold):
                indice.append(idx)
                count[arg[i]]+=1
            label.append(int(arg[i]))
            idx+=1

        # Filter the data and construct a new dataset.
    # print("ckpt 2: ",len(MyPseudoLabel))
    # # Turn off the eval mode.
    model.train()
    print("Labeled class distribution: ",count)
    print(f"Labeled / Unlabeled: {int(sum(count)):05d}/{idx:05d}")

    global MyPseudoLabel_unlabeled_set, tfm_unlabeled_set
    MyPseudoLabel_unlabeled_set = label
    tfm_unlabeled_set.target_transform = semi_unlabeled_set
    subset = Subset(tfm_unlabeled_set, indice)
    return subset

def get_pseudo_labels_1(dataset, model):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    model.eval()
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    indice = []
    label = []
    idx = 0
    count = torch.zeros((11))
    # print("ckpt 1: ",len(MyPseudoLabel))
    for batch in dataloader:
        img, _= batch
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        max_prob, arg = torch.max(probs,1)
        for i in range(len(probs)):
            if(max_prob[i] >= threshold):
                indice.append(idx)
                count[arg[i]]+=1
            label.append(int(arg[i]))
            idx+=1

    del dataloader
        # Filter the data and construct a new dataset.
    # print("ckpt 2: ",len(MyPseudoLabel))
    # # Turn off the eval mode.
    model.train()
    print("Labeled class distribution: ",count)
    print(f"Labeled / Unlabeled: {int(sum(count)):05d}/{idx:05d}")

    global MyPseudoLabel_test_set, tfm_test_set
    MyPseudoLabel_test_set = label
    tfm_test_set.target_transform = semi_test_set
    subset = Subset(tfm_test_set, indice)
    return subset


In [ ]:
# "cuda" only when GPUs are available.
# device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()
# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
# model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
# criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-5, momentum=0.9)
if Scheduler == "Plateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=64, eps=1e-12 ,verbose=True)
elif Scheduler == "Cosine":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=32, eta_min=1e-8)

best_acc = 0.0
last_update = 0
updated_acc = False
unlabeled_loader = None
test_unlabeled_loader = None


print("Start Training.")

for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if (best_acc >= 0.7) and updated_acc:
        do_semi = True
        updated_acc = False
    
    if do_semi:
        # Obtain pseudo-labels for unlabeled data using trained model.
        best_model = Classifier().to(device)
        best_model.load_state_dict(torch.load(model_path))
        pseudo_set = get_pseudo_labels_0(unlabeled_set, best_model)
        unlabeled_loader = DataLoader(pseudo_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

        test_pseudo_set = get_pseudo_labels_1(test_set, best_model)
        test_unlabeled_loader = DataLoader(test_pseudo_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

        del best_model
        do_semi = False

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    gradient_norm = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

Start Training.


100%|██████████| 25/25 [00:25<00:00,  1.03s/it]


[ Train | 001/4096 ] loss = 2.61705, acc = 0.11656


100%|██████████| 6/6 [00:05<00:00,  1.12it/s]


[ Valid | 001/4096 ] loss = 2.44146, acc = 0.08516


...

100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


[ Valid | 059/4096 ] loss = 1.48605, acc = 0.49870


100%|██████████| 25/25 [00:25<00:00,  1.00s/it]


[ Train | 060/4096 ] loss = 1.56059, acc = 0.46812


  0%|          | 0/6 [00:00<?, ?it/s]

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [11]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

100%|██████████| 27/27 [00:29<00:00,  1.10s/it]


In [12]:
# Save predictions into the file.
with open("tofu_predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")